# Hands-on session: Machine Learning & Data Visualization
---
## 5 - Data Visualization
---


How do I run this?

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sina-mansour/OHBM-Brainhack-2021)

Gitpod?

Binder?

---


## Scatter plots

---
## Line plots

---
## Histograms

---
## Subplots and overlays

---
## Neuroimaging visualizations